# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучим файл

### Подключим необходимые библиотеки.

In [91]:
import pandas as pd
import numpy as np

import random
from random import randint

import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

pd.set_option('display.max_columns', None)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.exceptions import DataConversionWarning
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

### Откроем таблицу, посмотрем внешний вид и изучим общую информацию о данных.

In [92]:
df = pd.read_csv('/datasets/users_behavior.csv')


display(df.head(10))
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### **Вывод:**
Предобработка данных не требуется.

## Разберем данные на выборки

### Разделим наш набор данных на  features (признаки) и target (целевой признак), где целевой признак это 'is_ultra' 

In [93]:
target = df['is_ultra']
features = df.drop('is_ultra', axis=1)
print('Размер target', target.shape[0])
print('Размер features', features.shape[0])

print('Соотношение меток 1 и 0 в таргете:', target[target == 0].count() / target[target == 1].count())      

Размер target 3214
Размер features 3214
Соотношение меток 1 и 0 в таргете: 2.2629441624365483


### Данные разобъем на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1

In [94]:
features_train, features_df, target_train, target_df = train_test_split(features, target
                                                                              , test_size=0.40
                                                                              , random_state=12345
                                                                              , stratify=target )
features_test, features_valid, target_test, target_valid = train_test_split(features_df, target_df
                                                                              , test_size=0.50
                                                                              , random_state=12345
                                                                              , stratify=target_df)

print('Размер обучающей выборки', features_train.shape[0])
print('Размер валидационной выборки', features_valid.shape[0])
print('Размер тестовой выборки', features_test.shape[0])

print('Соотношение меток 1 и 0 в target_valid:'
      , target_valid[target_valid == 0].count() / target_valid[target_valid == 1].count())
print('Соотношение меток 1 и 0 в target_test:'
      , target_test[target_test == 0].count() / target_test[target_test == 1].count())

print('Количество 0 в таргете:', target_test[target_test == 0].count())
print('Количество 1 в таргете:', target_test[target_test == 1].count())


Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643
Соотношение меток 1 и 0 в target_valid: 2.263959390862944
Соотношение меток 1 и 0 в target_test: 2.263959390862944
Количество 0 в таргете: 446
Количество 1 в таргете: 197


### **Вывод:**
* Сохранили соотношение меток 1 и 0 в таргете
* Данные готовы для исследования.

## Исследуем модели

### Модель классификатора дерева решений

Напишем блок, который перебором, меняя гиперпараметры ищет accuracy лучшей модели.

In [95]:
best_model = None
best_result = 0
for depth in range(1,50):
    for crit in ["gini", "entropy"]:
        for split in ["best", "random"]:
            model = DecisionTreeClassifier(random_state=12345
                                           , max_depth=depth
                                           , criterion=crit
                                           , splitter=split)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid) 
            result = accuracy_score(target_valid, predictions)
            print("Accuracy модели:", result
                  , 'при глубине дерева:', depth
                  , ', criterion=', crit
                  , ', splitter=', split ) 
            if result > best_result:
                best_split=split
                best_crit=crit
                max_depth=depth
                best_model = model
                best_result = result
        
print("Accuracy лучшей модели:", best_result
      , 'при глубине дерева:', max_depth
      , ', criterion=', best_crit
      , ', splitter=', best_split ) 

Accuracy модели: 0.7293934681181959 при глубине дерева: 1 , criterion= gini , splitter= best
Accuracy модели: 0.6936236391912908 при глубине дерева: 1 , criterion= gini , splitter= random
Accuracy модели: 0.7293934681181959 при глубине дерева: 1 , criterion= entropy , splitter= best
Accuracy модели: 0.6936236391912908 при глубине дерева: 1 , criterion= entropy , splitter= random
Accuracy модели: 0.7682737169517885 при глубине дерева: 2 , criterion= gini , splitter= best
Accuracy модели: 0.7153965785381027 при глубине дерева: 2 , criterion= gini , splitter= random
Accuracy модели: 0.7682737169517885 при глубине дерева: 2 , criterion= entropy , splitter= best
Accuracy модели: 0.7153965785381027 при глубине дерева: 2 , criterion= entropy , splitter= random
Accuracy модели: 0.7993779160186625 при глубине дерева: 3 , criterion= gini , splitter= best
Accuracy модели: 0.7247278382581649 при глубине дерева: 3 , criterion= gini , splitter= random
Accuracy модели: 0.7993779160186625 при глубине 

Сохраним перамеры в best_tree_model

In [96]:
best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=5, criterion="gini", splitter='best')
best_tree_model.fit(features_train, target_train)
predictions = best_tree_model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print("Accuracy лучшей модели:", result
      , 'при глубине дерева:', 5
      , ', criterion=', 'gini'
      , ', splitter=', 'best')

Accuracy лучшей модели: 0.8118195956454122 при глубине дерева: 5 , criterion= gini , splitter= best


### Модель случайного леса

Напишем блок, который перебором, меняя гиперпараметры ищет accuracy лучшей модели.

In [97]:
best_model = None
best_result = 0
for estim in range(1,50):
    for crit in ["gini", "entropy"]:
        for boot in [False, True]:
            for warm in [False, True]:
                model = RandomForestClassifier(random_state=12345
                                               , n_estimators=estim
                                               , criterion=crit
                                               , bootstrap=boot
                                               , warm_start=warm)
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid) 
                result = accuracy_score(target_valid, predictions)
                print("Accuracy ", result
                      , 'при количестве деревьев:', estim
                      , ', criterion=', crit
                      , ', bootstrap=', boot
                      , ', warm_start=', warm) 
                if result > best_result:
                    best_warm=warm
                    best_boot=boot
                    best_crit=crit
                    max_estim=estim
                    best_model = model
                    best_result = result
        
print("Accuracy лучшей модели:", best_result
      , 'при количестве деревьев:', max_estim
      , ', criterion=', best_crit
      , ', bootstrap=', best_boot
      , ', warm_start=', best_warm) 

Accuracy  0.7465007776049767 при количестве деревьев: 1 , criterion= gini , bootstrap= False , warm_start= False
Accuracy  0.7465007776049767 при количестве деревьев: 1 , criterion= gini , bootstrap= False , warm_start= True
Accuracy  0.7402799377916018 при количестве деревьев: 1 , criterion= gini , bootstrap= True , warm_start= False
Accuracy  0.7402799377916018 при количестве деревьев: 1 , criterion= gini , bootstrap= True , warm_start= True
Accuracy  0.7418351477449455 при количестве деревьев: 1 , criterion= entropy , bootstrap= False , warm_start= False
Accuracy  0.7418351477449455 при количестве деревьев: 1 , criterion= entropy , bootstrap= False , warm_start= True
Accuracy  0.7387247278382582 при количестве деревьев: 1 , criterion= entropy , bootstrap= True , warm_start= False
Accuracy  0.7387247278382582 при количестве деревьев: 1 , criterion= entropy , bootstrap= True , warm_start= True
Accuracy  0.7916018662519441 при количестве деревьев: 2 , criterion= gini , bootstrap= False

Сохраним параметры в best_forest_model

In [98]:
best_forest_model = RandomForestClassifier(random_state=12345
                                               , n_estimators=16
                                               , criterion="entropy"
                                               , bootstrap=True
                                               , warm_start=False)
best_forest_model.fit(features_train, target_train)
predictions = best_forest_model.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
            
print("Accuracy лучшей модели:", result
      , 'при количестве деревьев:', 16
      , ', criterion=', 'entropy'
      , ', bootstrap=', 'True'
      , ', warm_start=', 'False')

Accuracy лучшей модели: 0.8133748055987559 при количестве деревьев: 16 , criterion= entropy , bootstrap= True , warm_start= False


### Модель логистическая регрессия

Напишем блок, который перебором, меняя гиперпараметры ищет accuracy лучшей модели.

In [99]:
best_model = None
best_result = 0
for max_iter in range(71,73):
    for sol in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        for number_c in np.arange(0.1, 1.5, 0.1):
            for multi in ['auto', 'ovr']:
                model = LogisticRegression(random_state=12345
                                               , max_iter=max_iter
                                               , solver=sol
                                               , C=number_c
                                               , multi_class=multi)
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid) 
                result = accuracy_score(target_valid, predictions)
                print("Accuracy ", result
                      , 'при количестве итераций:', max_iter
                      , ', solver=', sol
                      , ', C=', number_c
                      , ', multi_class=', multi) 
                if result > best_result:
                    best_max_iter=max_iter
                    best_solver=sol
                    best_c=number_c
                    best_multi=multi
                    best_model = model
                    best_result = result
        
print("Accuracy лучшей модели:", best_result
      , 'при количестве итераций:', best_max_iter
      , ', solver=', best_solver
      , ', C=', best_c
      , ', multi_class=', best_multi) 


Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.1 , multi_class= auto
Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.1 , multi_class= ovr
Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.2 , multi_class= auto


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.2 , multi_class= ovr
Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.30000000000000004 , multi_class= auto


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.30000000000000004 , multi_class= ovr
Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.4 , multi_class= auto
Accuracy  0.7465007776049767 при количестве итераций: 71 , solver= newton-cg , C= 0.4 , multi_class= ovr


...

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= sag , C= 1.4000000000000001 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.1 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.1 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.2 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.2 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.30000000000000004 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.30000000000000004 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.4 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.4 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итерац

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.6 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.6 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.7000000000000001 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.7000000000000001 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.8 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.8 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.9 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 0.9 , multi_class= ovr
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver= saga , C= 1.0 , multi_class= auto
Accuracy  0.6936236391912908 при количестве итераций: 72 , solver

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

Сохраним параметры в best_logistic_model

In [100]:
best_logistic_model = LogisticRegression(random_state=12345
                                               , max_iter=77
                                               , solver='lbfgs'
                                               , C=1.0)                                                
best_logistic_model.fit(features_train, target_train)
predictions = best_logistic_model.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
print("Accuracy лучшей модели:", result
      , 'при количестве итераций:', 77
      , ', solver=', 'lbfgs'
      , ', C=', '1.0') 

Accuracy лучшей модели: 0.7573872472783826 при количестве итераций: 77 , solver= lbfgs , C= 1.0


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### **Вывод:**
**Лучшими параметрами оказались:**
* Для классификатора дерева решений Accuracy лучшей модели: 0.8118195956454122 при глубине дерева: 5 , criterion= gini , splitter= best
* Для модели случайного леса Accuracy лучшей модели: 0.8133748055987559 при количестве деревьев: 16 , criterion= entropy , bootstrap= True , warm_start= False
* Для логистической регрессии Accuracy лучшей модели: 0.7573872472783826 при количестве итераций: 72 , solver= lbfgs , C= 1.0
* По наибольшему значению Accuracy выберакем модель случайного леса.

## Проверим модели на тестовой выборке

Проверим модели на тестовой выборке и найдем сразу точность, полноту и F1-меру для визуализации результатов в следующем пункте. 

In [101]:
test_predictions = best_forest_model.predict(features_test)
result_forest = accuracy_score(target_test, test_predictions)
print('accuracy модели случайного леса на тестовой выборке', result_forest)

precision_forest = precision_score(target_test, test_predictions)
recall_forest = recall_score(target_test, test_predictions)
f1_forest = f1_score(target_test, test_predictions)


accuracy модели случайного леса на тестовой выборке 0.7931570762052877


### **Вывод:**
**На тестовой выборке для модели случайного леса accuracy  =  0.793, является отличным результатом.**

## (бонус) Проверьте модели на адекватность

Создадим список состоящий из 446 нулей и 197 единиц, чтоб учесть дисбаланс классов в таргете, и перемешаем его методом random.shuffle. 

In [102]:
rand_0 = [0]
rand_1 = [1]
rand_0 = rand_0 * 446
rand_1 = rand_1 * 197
rand = rand_0 + rand_1
random.shuffle(rand)
rand_series = pd.Series(rand)
print(rand_series.value_counts())

result_rand= accuracy_score(target_test, rand_series)
print('Точность модели случайного леса на тестовой выборке', result_rand) 

0    446
1    197
dtype: int64
Точность модели случайного леса на тестовой выборке 0.5987558320373251


Сформируем сводную таблицу-отчет отражающую качество моделей на тестовой выборке в сравнении моделью, где результат предсказывается случайным образом.

### **Вывод:**
* По сравнению с моделью случайного предсказания у модели случайного леса качество выше на 34%, т. е. в полне адекватна. 
* **Если учесть, что метрика качество у случайного леса 0.79 то можно порекомендовать эту модель для системы способной проанализировать поведение клиентов.**	

**Общий вывод:**

Данные были разбиты на три части: обучающую, валидационную и тестовую выборки, в соотношении 3:1:1.
Получились:
* Размер обучающей выборки 1928
* Размер валидационной выборки 643
* Размер тестовой выборки 643

Провели исследование модели:
* классификатора дерева решений
* случайного леса
* логистической регрессии

**Лучшими параметрами оказались:**
* Для классификатора дерева решений Accuracy лучшей модели: 0.8118195956454122 при глубине дерева: 5 , criterion= gini , splitter= best
* Для модели случайного леса Accuracy лучшей модели: 0.8133748055987559 при количестве деревьев: 16 , criterion= entropy , bootstrap= True , warm_start= False
* Для логистической регрессии Accuracy лучшей модели: 0.7573872472783826 при количестве итераций: 72 , solver= lbfgs , C= 1.0
* По наибольшему значению Accuracy выберакем модель случайного леса.

**На тестовой выборке для модели случайного леса accuracy  =  0.793, является отличным результатом.**

* Модель случайного леса с мерой адекватности F1 равна 0.619, это выше среднего значения адекватности модели. 

* **Если учесть, что метрика accuracy у  случайного леса 0.793 и средняя адекватность то можно порекомендовать эту модель для системы способной проанализировать поведение клиентов.**				